In [22]:
class StackingEstimator():
    
    def __init__(self, models, cvs, n_iterations, subsamples, seed):
        '''
        models - список с ансамблем моделей
        cvs - список с схемами валидации
        n_iterations - итераций стекинга
        subsamples - список с долями используемых признаков 
        seed - генератор случайных чисел
        '''        
        self.models = models
        self.cvs = cvs
        self.n_iterations = n_iterations
        self.subsamples = subsamples
        self.seed = seed
    def fit(self, X, y):
        '''
        1) обучаем модели на валидации
        2) сохраняем обученные модели
        '''        
        try:
            _X, _y = np.array(X), np.array(y)
        except:
            _X, _y = X, y  
        nfeatures = _X.shape[1]
        feat_idxs = np.arange(nfeatures)
        estimators_use_idxs = []        
        for iteration in range(self.n_iterations):
            for model_idx, model in enumerate(models):
                for subsample_idx, subsample in subsamples:
                    np.random.seed(seed+iteration+model_idx+subsample_idx+3)
                    use_idxs = np.random.choice(feat_idxs, np.int32(np.around(subsample*nfeatures)))
                    for cv in cvs:
                        for tr_idx, val_idx in cv.split(_y):
                            model.fit(_X[tr_idx, use_idxs], _y[tr_idx])
                            estimators_use_idxs.append((model, use_idxs))        
        self.estimators_use_idxs = estimators_use_idxs
    def get_metafeatures(self, X):
        '''
        с помощью обученных моделей получаем метапризнаки
        '''
        try:
            _X = np.array(X)
        except:
            _X = X
        L = []    
        for use_idxs, estimator in self.estimators_use_idxs:
            try:
                L.append(estimator.predict_proba(_X[:, use_idxs])[:, 1])
            except:
                L.append(estimator.predict(_X[:, use_idxs]))
                
        return np.column_stack(L)